# CAMS Animation

Notebook to generate a GIF of CAMS plots for a period and extent of your choosing

In [ ]:
import os, sys, glob

# Adds the package path to the Python path to make sure all the local imports work fine 
if os.path.dirname(os.path.dirname(os.getcwd())) not in sys.path:
    sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

# local imports
from constants import POLLUTANTS, DATA_DIR_CAMS_AN, DATA_DIR_PLOTS

# import remaining packages needed for the script
import matplotlib.pyplot as plt
from datetime import datetime
import cartopy.crs as ccrs
from PIL import Image
import xarray as xr
from pathlib import Path

## Set the paramaters

In [ ]:
pollutants = [
    'CO',
    'O3',
    'NO',
    'NO2',
    'PM25',
    'PM10',
    'SO2',
]

period = {
    'start_date':datetime(year=2021, month=4, day=20, hour=12),
    'end_date':datetime(year=2021, month=4, day=21, hour=12),
}

extent = {
    'max latitude': 55.65,  
    'min longitude': -11.35, 
    'min latitude': 51.25,   
    'max longitude': -5.35,  
}

# See this page for the available colormaps: https://matplotlib.org/stable/gallery/color/colormap_reference.html
colormap = 'RdYlGn_r'

# folder where the final GIF will be stored
output_folder = Path(DATA_DIR_PLOTS).joinpath('animation')

# create the output folder if it does not exist yet
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

## Process the data

In [ ]:
datasets = {}

for pollutant in pollutants:
    
    # Open CAMS dataset
    pollutant_cams_name = POLLUTANTS[pollutant]['CAMS']
    
    ds = xr.open_dataset(Path(DATA_DIR_CAMS_AN).joinpath(f'{pollutant_cams_name}.nc'))

    # select cams data within the selected time period
    ds = ds.sel(time=slice(period['start_date'], period['end_date']))
    
    # select cams data within the selected extent
    min_lon = extent['min longitude']
    min_lat = extent['min latitude']
    max_lon = extent['max longitude']
    max_lat = extent['max latitude']
    
    mask_lon = (ds.longitude >= min_lon) & (ds.longitude <= max_lon)
    mask_lat = (ds.latitude >= min_lat) & (ds.latitude <= max_lat)
    ds = ds.where(mask_lon & mask_lat, drop=True)
    
    # store the dataset in the dictionary
    datasets[pollutant] = ds

## Generate the plots & GIF

In [ ]:
for pollutant in datasets:
    
    pollutant_full_name = POLLUTANTS[pollutant]['FULL_NAME']
    pollutant_formula = POLLUTANTS[pollutant]['FORMULA']
    pollutant_cams_name = POLLUTANTS[pollutant]['CAMS']
    
    output_folder_pol = output_folder.joinpath(pollutant_cams_name)
    
    if not os.path.exists(output_folder_pol):
            os.makedirs(output_folder_pol)
    
    ds = datasets[pollutant][pollutant_cams_name]
    
    min_val = ds.min().data
    max_val = ds.max().data

    for i in ds.time.to_pandas().items():
        i = i[0]
        hour = ds.sel(time = i)

        hour = hour.assign_attrs(
                {'long_name': f'{pollutant_full_name} Concentration',
                 'units': 'µg m-3',}
            )

        fig, ax = plt.subplots(figsize=(10,10))
        ax = plt.axes(projection=ccrs.Orthographic(8.7, 49.9))
        ax.coastlines()

        hour.plot(
            transform=ccrs.PlateCarree(),
            robust=True,
            vmin=min_val,
            vmax=max_val,
            cmap=plt.get_cmap(colormap),
            extend='neither',
            facecolor="gray"
        )

        title_name = i.strftime("%H:%M %d %b %Y")
        ax.set_title(f'{pollutant_full_name} ({pollutant_formula}) {title_name}',
                            {'fontsize': 18})

        plt.savefig(output_folder_pol.joinpath(f'{i.strftime("%Y_%b_%d_%H_%M")}.jpg'))
        plt.close()
    
    fp_in = f'{output_folder_pol.as_posix()}/*.jpg'
        
    # https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
    img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
            
    img.save(
        fp=output_folder_pol.joinpath(f'animation_{period["start_date"].strftime("%d_%b_%Y")}_to_{period["end_date"].strftime("%d_%b_%Y")}.gif'),
        format='GIF',
        append_images=imgs,
        save_all=True,
        duration=600,
        loop=0,
    )
    
    for f in glob.glob(fp_in):
        try:
            os.remove(f)
        except OSError as e:
            print("Error: %s : %s" % (f, e.strerror))